In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from hyperopt import tpe
import numpy as np

In [3]:
df = pd.read_csv('clean_data.csv')

In [4]:
pdf = df.reindex(np.random.permutation(df.index))

In [5]:
X, y = pdf['clean_text'], pdf['class']

In [6]:
vec = CountVectorizer(ngram_range=(1, 3))
X_enc = vec.fit_transform(X)

In [7]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=.33)

In [9]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
accuracy_score(y_test, lr.predict(X_test))

/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8196969696969697

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
from sklearn.metrics import f1_score

def f1_loss(y_target, y_pred):
    f1 = f1_score(y_target, y_pred)
    print(f"this loss called with f1={f1}")
    return 1.0 - f1


from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing, any_sparse_classifier, sgd

np.random.seed(13)

estim = HyperoptEstimator(classifier=sgd('yes'),
                          preprocessing=[],
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120,
                         loss_fn=f1_loss, verbose=False)

# Search the hyperparameter space based on the data

estim.fit(X_enc, y, random_state=42, cv_shuffle=False, n_folds=None, )

# Show the results

print(estim.score(X_enc, y))
# 1.0

print( estim.best_model() )

Will use the last 0.2 portion of samples for validation
Training learner SGDClassifier(alpha=3.801113913337704e-05, average=False,
              class_weight='balanced', early_stopping=False, epsilon=0.1,
              eta0=0.01297891019764783, fit_intercept=True,
              l1_ratio=0.6287951512009251, learning_rate='constant',
              loss='squared_hinge', max_iter=30148914.0, n_iter_no_change=5,
              n_jobs=1, penalty='l1', power_t=0.10140007388288363,
              random_state=2, shuffle=True, tol=0.00448603330673893,
              validation_fraction=0.1, verbose=False, warm_start=False) on X/EX of dimension (1600, 1236908)
Scoring on X/EX validation of shape (400, 1236908)
this loss called with f1=0.6842105263157895
OK trial with loss 0.3
100%|██████████| 1/1 [00:02<00:00,  2.58s/it, best loss: 0.3157894736842105]
Will use the last 0.2 portion of samples for validation
Training learner SGDClassifier(alpha=2.9479784086768314e-06, average=False,
              cla

Process Process-126:

Traceback (most recent call last):

  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()

  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)

  File "/home/ptyshevskyi/Desktop/UCU/hyperopt-sklearn/hpsklearn/estimator.py", line 326, in _cost_fn
    learner.fit(XEXfit, yfit)

  File "/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py", line 713, in fit
    sample_weight=sample_weight)

  File "/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py", line 554, in _fit
    classes, sample_weight, coef_init, intercept_init)

  File "/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py", line 514, in _partial_fit
    max_iter=max_iter)

  File "/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linea

In [22]:
model = estim.best_model()['learner']
cross_val_score(model, X_enc, y, cv=5).mean()

0.5439999999999999

In [14]:
model

SGDClassifier(alpha=0.0026461806301647244, average=False,
              class_weight='balanced', early_stopping=False, epsilon=0.1,
              eta0=3.3361348224900434e-05, fit_intercept=True,
              l1_ratio=0.03460388488308809, learning_rate='constant',
              loss='huber', max_iter=15606531.0, n_iter_no_change=5, n_jobs=1,
              penalty='elasticnet', power_t=0.017345142216377618,
              random_state=2, shuffle=True, tol=0.0002393463072572627,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [18]:
cross_val_score(SGDClassifier(random_state=42), X_enc, y).mean()

0.8195051338763913